In [3]:
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
import json
import pandas as pd

load_ids=json.load(open('ids.json','r'))
loaded_ids=load_ids#.read()
# load_ids.close()

api_key = loaded_ids["api"]
clientId = loaded_ids["c_id"]
pwd = loaded_ids['pwd']
smartApi = SmartConnect(api_key)
smartApi._routes["api.market.data"]="/rest/secure/angelbroking/market/v1/quote"
token = loaded_ids["totp"]
totp=pyotp.TOTP(token).now()
correlation_id = "hdhdjdjdjd"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

In [7]:
from datetime import datetime,timedelta,date
import time
import talib
import numpy as np
start=time.time()

def history(historicParam):
    try:
        return smartApi.getCandleData(historicParam)
    except Exception as e:
        print("Historic Api failed: {}".format(e.message))



t=datetime.now()
#df=datetime(t.year,t.month,t.day,9,15)
t=datetime(t.year,t.month,t.day,t.hour,t.minute)
t=t+timedelta(hours=5,minutes=30)
#print(t)
arr=[]

historicParam={    "exchange": "NSE",
    "symboltoken": "3045",
    "interval": "ONE_MINUTE",
    "fromdate": "",
    "todate": ""
    }
fromdate=datetime(t.year,t.month,t.day,9,15)
fromdate=fromdate-timedelta(days=5)
todate=t
#print(fromdate,' ',todate)
historicParam['fromdate']=fromdate.strftime("%Y-%m-%d %H:%M")
historicParam['todate']=todate.strftime("%Y-%m-%d %H:%M")

hist=history(historicParam)
#print(hist)
hist=hist['data']
print(len(hist))
columns=['timestamp','open','high','low','close','volume']
data=pd.DataFrame(hist,columns=columns)
data['timestamp']=[i.replace('T',' ')[0:-6] for i in data['timestamp']]
data['timestamp']=pd.to_datetime(data['timestamp'],format="%Y-%m-%d %H:%M:%S")
val={}
Columns=['timestamp','close']
for col in Columns:
    val[col]=np.array(data[col]).flatten()
ind=np.array(data.index).flatten()
df=pd.DataFrame(val,index=ind)
#sc=StandardScaler()
#df[Columns]=sc.fit_transform(df[Columns])
#for i in range(len(df)):
    #df['close'][i]=df['close'][i]*0.001
print(time.time()-start)

1500
0.2995467185974121


In [53]:
data.head()

,timestamp,open,high,low,close,volume
0,2024-02-15 09:15:00,747.70,747.70,743.40,744.50,340459
1,2024-02-15 09:16:00,744.80,744.80,742.55,742.90,288985
2,2024-02-15 09:17:00,742.90,746.00,742.65,745.45,237369
3,2024-02-15 09:18:00,745.15,747.35,745.15,746.20,243458
4,2024-02-15 09:19:00,746.00,746.50,744.70,745.50,212521


In [54]:
t=datetime.now()
t=datetime(t.year,t.month,t.day,9,16)
fromdate=t#-timedelta(minutes=1)
todate=t
historicParam['fromdate']=fromdate.strftime("%Y-%m-%d %H:%M")
historicParam['todate']=todate.strftime("%Y-%m-%d %H:%M")

hist=history(historicParam)['data']
print(hist)

[['2024-02-20T09:16:00+05:30', 756.0, 756.9, 755.6, 756.45, 58719]]


In [32]:
from tensorflow.keras.models import Sequential, model_from_json
json_file = open('SBIN_model_1min_97.json', 'r')
loaded_model_json = json_file.read()
# json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("SBIN_model_1min_97.h5")
print("Loaded model from disk")

Loaded model from disk


In [33]:
_7EMA=talib.EMA(np.array(df['close']),7)
_21EMA=talib.EMA(np.array(df['close']),20)

In [55]:

buy=0
sell=0
loss=0
k=1
last_7EMA=_7EMA[-1]
last_21EMA=_21EMA[-1]
last_timestamp=df['timestamp'][len(df)-1]
org=np.array(df['close'][-5:]).flatten()
x=1
while True:
    time.sleep(2.5)
    # for testing purpose
    # Fetching latest interval data
    t=datetime.now()
    t=datetime(t.year,t.month,t.day,9,15)
    t=t+timedelta(minutes=x)
    fromdate=t
    todate=t
    #######################
    x+=1
    #######################
    historicParam['fromdate']=fromdate.strftime("%Y-%m-%d %H:%M")
    historicParam['todate']=todate.strftime("%Y-%m-%d %H:%M")

    present=history(historicParam)['data']
    print(present)
    columns=['timestamp','open','high','low','close','volume']
    present=pd.DataFrame(present,columns=columns)

    present['timestamp']=[i.replace('T',' ')[0:-6] for i in present['timestamp']]

    present['timestamp']=pd.to_datetime(present['timestamp'],format="%Y-%m-%d %H:%M:%S")
    # print(present)
    if len(present)>0 and last_timestamp!=present['timestamp'][0]:
        l=last_7EMA
        r=last_21EMA
        last_timestamp=present['timestamp'][0]
        #print(last_timestamp)
        last_7EMA=0.25*present['close'][0]+0.75*last_7EMA
        last_21EMA=(2/21)*present['close'][0]+(1-2/21)*last_21EMA
        org=org[1:]
        org=np.append(org,present['close'][0])
        for k in range(5):
            org[k]=org[k]*0.001

        org=org.reshape((1,5,))

        pred_next=loaded_model.predict(org)*1000+0.661

        EMA7=0.25*pred_next+0.75*last_7EMA
        EMA21=(2/21)*pred_next+(1-2/21)*last_21EMA
        print(present['timestamp'][0],' ',present['close'][0],' ',last_7EMA,' ',last_21EMA,' Pred:',pred_next)
        if (last_7EMA<last_21EMA and EMA7>EMA21) or (l<r and last_7EMA>last_21EMA):
            angle1=(EMA7-last_7EMA)-(EMA21-last_21EMA)
            angle2=(last_7EMA-l)-(last_21EMA-r)
            if angle1>0.13 or angle2>0.13:
                print('BOUGHT AT:',last_timestamp)
        org=np.array(org).flatten()
        for k in range(5):
            org[k]=org[k]*1000
#     continue
#     #break
#     for j in range(5):
#         org[j]=org[j]*0.001
#         #print(j)
#     org=org.reshape((1,5,))
#     #pred_next=mod.predict(org)
#     #print(df['close'][i+1],' ',pred_next*1000)
#     pred_7EMA=_7EMA[i+1]
#     pred_21EMA=_21EMA[i+1]
#     if _7EMA[i]<_21EMA[i] and pred_7EMA>pred_21EMA:
#         angle=(pred_7EMA-_7EMA[i])-(pred_21EMA-_21EMA[i])
#         #print(angle,' ',df['timestamp'][i])
#         if angle>0.13:
#             times.append([df['timestamp'][i],_7EMA[i],_21EMA[i],pred_7EMA,pred_21EMA])
# #output.clear()
# for i in times:
#     print(i)

[['2024-02-20T09:16:00+05:30', 756.0, 756.9, 755.6, 756.45, 58719]]
1/1 [==============================] - 0s 26ms/step
2024-02-20 09:16:00   756.45   759.3111954374435   759.7633641713076  Pred: [[758.35443]]
[['2024-02-20T09:17:00+05:30', 756.35, 756.55, 755.55, 755.55, 65657]]
1/1 [==============================] - 0s 38ms/step
2024-02-20 09:17:00   755.55   758.3708965780827   759.3620913930879  Pred: [[757.03894]]
[['2024-02-20T09:18:00+05:30', 755.8, 755.8, 754.2, 754.35, 80353]]
1/1 [==============================] - 0s 37ms/step
2024-02-20 09:18:00   754.35   757.365672433562   758.884749355651  Pred: [[755.5187]]
[['2024-02-20T09:19:00+05:30', 754.35, 754.65, 754.05, 754.65, 55629]]
1/1 [==============================] - 0s 24ms/step
2024-02-20 09:19:00   754.65   756.6867543251716   758.4814398932081  Pred: [[754.4042]]
[['2024-02-20T09:20:00+05:30', 754.7, 755.7, 754.6, 755.05, 71204]]
1/1 [==============================] - 0s 35ms/step
2024-02-20 09:20:00   755.05   756.277